In [2]:
import pandas as pd 
import json

In [2]:
import json
import ast

dict_list = []

with open("data/review-Alaska.json", "r", encoding="utf-8") as f:
    for line_num, line in enumerate(f, start=1):
        line = line.strip()  # remove leading/trailing whitespace
        if not line:
            continue  # skip empty lines
        try:
            # Try JSON parsing first (double quotes)
            obj = json.loads(line)
        except json.JSONDecodeError:
            try:
                # Fallback for Python dict strings (single quotes)
                obj = ast.literal_eval(line)
            except Exception:
                print(f"Skipping invalid line {line_num}: {line[:100]}...")
                continue
        dict_list.append(obj)

print(f"Total valid entries: {len(dict_list)}")



    

Total valid entries: 1051246


In [4]:
df_reviews = pd.DataFrame(dict_list)

In [5]:
df_reviews = df_reviews.dropna(subset=['text']).reset_index(drop=True)


In [7]:
import json
import ast

dict_list = []

with open("data/meta-Alaska.json", "r", encoding="utf-8") as f:
    for line_num, line in enumerate(f, start=1):
        line = line.strip()  # remove leading/trailing whitespace
        if not line:
            continue  # skip empty lines
        try:
            # Try JSON parsing first (double quotes)
            obj = json.loads(line)
        except json.JSONDecodeError:
            try:
                # Fallback for Python dict strings (single quotes)
                obj = ast.literal_eval(line)
            except Exception:
                print(f"Skipping invalid line {line_num}: {line[:100]}...")
                continue
        dict_list.append(obj)

print(f"Total valid entries: {len(dict_list)}")

Total valid entries: 12774


In [8]:
df_meta = pd.DataFrame(dict_list)

In [9]:
merged_df = pd.merge(df_reviews, df_meta, on='gmap_id', how='inner')

In [10]:
merged_df = merged_df.drop_duplicates(subset=['name_x','text','gmap_id'])

In [11]:
merged_df.columns

Index(['user_id', 'name_x', 'time', 'rating', 'text', 'pics', 'resp',
       'gmap_id', 'name_y', 'address', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC',
       'state', 'relative_results', 'url'],
      dtype='object')

In [12]:
final_columns = ['text','resp','description','category','name_y']
final_df = merged_df[final_columns]

In [13]:
for col in final_df.columns:
    final_df[col] = final_df[col].apply(
        lambda x: ", ".join(x) if isinstance(x, list) else x
    )


C:\Users\chunw\AppData\Local\Temp\ipykernel_17236\803837385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[col] = final_df[col].apply(


In [14]:
num_nulls = final_df['description'].isna().sum()
print("Number of nulls in 'description':", num_nulls)

Number of nulls in 'description': 356678


In [20]:
# Reset index so you get a new incrementing column
final_df = final_df.reset_index(drop=True)

# Rename that index column
final_df.index.name = "id"

In [23]:
final_df = final_df.rename(columns={
    "name_y": "BusinessName"
})


In [24]:
final_df

,text,resp,description,category,BusinessName
id,,,,,
0,We always stay here when in Valdez for silver ...,None,None,"RV park, Cabin rental agency, Campground",Bear Creek Cabins & RV Park
1,This was an amazing RV camping experience with...,None,None,"RV park, Cabin rental agency, Campground",Bear Creek Cabins & RV Park
2,Spent the summer of 2011. Had a wonderful time...,None,None,"RV park, Cabin rental agency, Campground",Bear Creek Cabins & RV Park
3,My Wife and I have stayed at Bear Creek severa...,None,None,"RV park, Cabin rental agency, Campground",Bear Creek Cabins & RV Park
4,Great campground for the price. Nice hot unlim...,None,None,"RV park, Cabin rental agency, Campground",Bear Creek Cabins & RV Park
...,...,...,...,...,...
629195,(Translated by Google) Very popular route for ...,None,None,Shopping mall,Ketchikan Plaza
629196,(Translated by Google) small picturesque A.eri...,None,None,Shopping mall,Ketchikan Plaza
629197,"(Translated by Google) Umerenno, ne ahti\n\n(O...",None,None,Shopping mall,Ketchikan Plaza


In [26]:
final_df["user_message"] = (
    "Business Name: " + final_df["BusinessName"].astype(str).fillna("N/A") + "\n"
    "Category: " + final_df["category"].astype(str).fillna("N/A") + "\n"
    "Description: " + final_df["description"].astype(str).fillna("N/A") + "\n"
    "Review: " + final_df["text"].astype(str).fillna("N/A") + "\n"
    "Response: " + final_df["resp"].astype(str).fillna("N/A")
)



In [27]:
final_df

,text,resp,description,category,BusinessName,user_message
id,,,,,,
0,We always stay here when in Valdez for silver ...,None,None,"RV park, Cabin rental agency, Campground",Bear Creek Cabins & RV Park,Business Name: Bear Creek Cabins & RV Park\nCa...
1,This was an amazing RV camping experience with...,None,None,"RV park, Cabin rental agency, Campground",Bear Creek Cabins & RV Park,Business Name: Bear Creek Cabins & RV Park\nCa...
2,Spent the summer of 2011. Had a wonderful time...,None,None,"RV park, Cabin rental agency, Campground",Bear Creek Cabins & RV Park,Business Name: Bear Creek Cabins & RV Park\nCa...
3,My Wife and I have stayed at Bear Creek severa...,None,None,"RV park, Cabin rental agency, Campground",Bear Creek Cabins & RV Park,Business Name: Bear Creek Cabins & RV Park\nCa...
4,Great campground for the price. Nice hot unlim...,None,None,"RV park, Cabin rental agency, Campground",Bear Creek Cabins & RV Park,Business Name: Bear Creek Cabins & RV Park\nCa...
...,...,...,...,...,...,...
629195,(Translated by Google) Very popular route for ...,None,None,Shopping mall,Ketchikan Plaza,Business Name: Ketchikan Plaza\nCategory: Shop...
629196,(Translated by Google) small picturesque A.eri...,None,None,Shopping mall,Ketchikan Plaza,Business Name: Ketchikan Plaza\nCategory: Shop...
629197,"(Translated by Google) Umerenno, ne ahti\n\n(O...",None,None,Shopping mall,Ketchikan Plaza,Business Name: Ketchikan Plaza\nCategory: Shop...


In [28]:
final_df.to_csv("data.csv", index=True)  


In [3]:
df = pd.read_csv("data.csv")


In [4]:
print(df['user_message'][0])

Business Name: Bear Creek Cabins & RV Park
Category: RV park, Cabin rental agency, Campground
Description: None
Review: We always stay here when in Valdez for silver salmon fishing. The elderly couple that run it are amazing to talk to, extremely helpful. The campsites are very well maintained.
Response: None


In [5]:
from openai import OpenAI
import json
import pandas as pd
import os
from dotenv import load_dotenv

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)


In [6]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)  # force overwrite existing environment variables

load_dotenv()  # loads .env variables
api_key = os.environ.get("OPENAI_API_KEY")
print("API key loaded:", bool(api_key))  # just to check


API key loaded: True


In [55]:
import os
print(os.getcwd())

d:\rudys-rangers-techjam25


In [ ]:
system_message = '''
 <task>
    Analyze Google reviews to gauge review quality by detecting problematic content.
  </task>
  
  <detection_criteria>
    <spam>Detect reviews that are fake, duplicate, or artificially generated</spam>
    <advertisement>Detect reviews that primarily promote other businesses or services</advertisement>
    <irrelevant_content>Detect reviews that don't relate to the actual business or location</irrelevant_content>
    <non_visitor_rant>Detect rants from users who likely never visited the location</non_visitor_rant>
    <toxicity>Detect reviews containing rudeness, racism, hate speech, harassment, or other toxic behavior</toxicity>
  </detection_criteria>
  
  <labeling_instructions>
    Label each detection category with 1 if the issue is present, 0 if not present.
  </labeling_instructions>
  
  <input_format>
    Review text will be provided for analysis.
  </input_format>
  
  <response_format>
    {
      "spam": 0 or 1,
      "advertisement": 0 or 1, 
      "irrelevant_content": 0 or 1,
      "non_visitor_rant": 0 or 1,
      "toxicity": 0 or 1
      "reasoning": 
    }
  </response_format>
  
  <example>
    Input: "This place is terrible! I heard from my friend it's dirty and overpriced. Never going there!"
    Output: {"spam": 0, "advertisement": 0, "irrelevant_content": 0, "non_visitor_rant": 1, "toxicity": 0}
  </example>
  <example>
    Input: "Had dinner here last night with my family. The food was okay I guess, service was a bit slow. But speaking of great service, I have to mention that 
    I recently had my car detailed by Elite Auto Spa on Main Street and it was absolutely incredible! They made my 10-year-old Honda look brand new. 
    The owner Dave is super friendly and they use eco-friendly products. Only $89 for a full detail - such a steal! If anyone needs their car cleaned, 
    definitely check out Elite Auto Spa. They're open 7 days a week and take walk-ins. Tell them Mike sent you for 10% off your first visit!
    Anyway, the restaurant was fine. Might come back sometime."
    Output:  {"spam": 0, "advertisement": 1, "irrelevant_content": 1, "non_visitor_rant": 0, "toxicity": 0}
  </example>
  <example>
    Input: "This place is absolutely DISGUSTING and the staff are complete idiots who don't know anything about customer service!!! The manager Karen is a total b**** who needs to be fired immediately. Food gave me food poisoning and I'm going to sue them for everything they're worth.
    EVERYONE NEEDS TO BOYCOTT THIS TERRIBLE RESTAURANT. They are probably money laundering and breaking health codes. I saw mexican ROACHES in the kitchen (I didn't but whatever).
    By the way, if you want REAL Italian food, go to Tony's Pizza Palace downtown - they have the BEST pizza in the city and Maria the owner is amazing! Use code FOODIE20 for 20% off! They're having a grand opening special this week!
    Also check out my food blog at totally-real-food-reviews.net where I expose restaurants like this one. Follow me on Instagram @foodcritic_real for more honest reviews!
    DO NOT EAT HERE - YOU HAVE BEEN WARNED!!!"
    Output: {"spam": 0, "advertisement": 0, "irrelevant_content": 1, "non_visitor_rant": 0, "toxicity": 1}
  </example>  
'''

In [ ]:
from pydantic import BaseModel
from typing import List
import json
import os
import pandas as pd
from openai import OpenAI


class MessageInfo(BaseModel):
    spam: str
    advertisement: str
    irrelevant_content: str
    non_visitor_rant: str
    toxicity:str 

class Messages(BaseModel):
    messages: List[MessageInfo]

class BatchProcess():
    def __init__(self, df:pd.DataFrame, start_index: int, batch_start_number:int, system_message:str, client:OpenAI):
        """ 
        prepares raw jd from df, sends it to llms and writes outputs to json
        
        """
        self.df = df
        self.start_index = start_index #index of the dataframe to START
        self.batch_start_number = batch_start_number #batch number to START
        self.system_message = system_message
        self.client = client

    def process(self, no_of_batches:int, batch_size:int) -> None:
        #prepare the messages in the batch
        for i in range(no_of_batches):
            print(f"Processing batch {i}...")
            
            messages, row_ids, checkpoint = self.prepare_llm_inputs(self.df, self.start_index, batch_size) #process before llm
            llm_structured_output = self.get_structured_output_from_llm(self.system_message, messages) #pass to llm, get responses

            print("Processing row_ids...", row_ids)

            if len(llm_structured_output.messages) != len(row_ids):
                print(messages)
                print(llm_structured_output.messages)
                raise ValueError(f"Error: Expected batch size of {batch_size}, but got jobids: {len(row_ids)} and llmoutputs: {len(llm_structured_output.messages)}.")

            batch_jds = self.append_jb_ids(row_ids, llm_structured_output) #process output, append jd ids to the response

            self.write_to_file(batch_jds, checkpoint, self.batch_start_number) #write to json for the batch
            
            self.start_index = checkpoint

            self.batch_start_number += 1
        


       
    # need to iterate through the rows in the df for that batch from the start checkpoint, 
    def prepare_llm_inputs(self, df: pd.DataFrame, checkpoint:int, batch_size:int) -> tuple[str, List[int], int]:
        """ 
        This function iterates through the rows in the dataframe in that batch and prepares multiple raw JDs into one message/string for the llm
        returns the updated checkpoint, input prompt which is a string and the list of their job ids for addition later
        """
        messages =""
        job_ids = []
        counter = 1
        for i in range(checkpoint, checkpoint+batch_size):
            job_id = int(df.iloc[i]['id'])
            raw_jd = df.iloc[i]['user_message'].strip()
            job_ids.append(job_id)
            messages += f"<Job {counter}>\n{raw_jd}\n</Job {counter}>\n\n"
            counter+=1
        # Check if the number of job ids does not match the batch size
        if len(job_ids) != batch_size:
            
            raise ValueError(f"Error: Expected batch size of {batch_size}, but got {len(job_ids)}.")

        return (messages, job_ids, checkpoint + batch_size)
    

            
    def get_structured_output_from_llm(self, system_message: str, user_message:str) -> Messages:
        """ 
        sends the messages to gemini, returns a Messages object
        """
        response = self.client.beta.chat.completions.parse(
            model="gemini-2.0-flash-lite",
            messages=[
                {"role": "system", "content": system_message},
                {
                    "role": "user",
                    "content": user_message
                }
            ],
            response_format=Messages,
        )
        return response.choices[0].message.parsed
    
    # we need to append the job ids to each message, saves output to json file
    def append_jb_ids(self, job_ids:List, messages_list: Messages) -> List:
        batch_jds = []
        for idx, message in enumerate(messages_list.messages):
            temp = message.model_dump()
            #print(temp, idx)
            temp['job_id'] = job_ids[idx]
            batch_jds.append(temp)

        return batch_jds
    
    def write_to_file(self, batch_jds:List, row_checkpoint, batch_no) -> None:
            row_checkpoint -= 1
            dir_name = "extracted_jds"
            os.makedirs(dir_name, exist_ok=True)

            filename = f"{dir_name}/batch_{batch_no}_row_{row_checkpoint}_extracted_jd.json"

            with open(filename, "w") as f:
                json.dump(batch_jds, f, indent=4) 


In [42]:
fake = [{"id":0, "user_message":"my friend told me this place sucks!!!"}, {"id":1, "user_message":"Escape the city—relax, explore, and unwind at Bear Creek Cabins & RV Park! 50% off for new users!"}]
fake_df = pd.DataFrame(fake)

In [43]:
fake_df

,id,user_message
0,0,my friend told me this place sucks!!!
1,1,"Escape the city—relax, explore, and unwind at ..."


In [49]:
batch = BatchProcess(fake_df, 0, 0, system_message, client)

In [50]:
messages, job_ids, checkpoint = batch.prepare_llm_inputs(fake_df, 0, 2)

In [51]:
response = batch.get_structured_output_from_llm(system_message, messages)

In [52]:
response

Messages(messages=[MessageInfo(spam='0', advertisement='0', irrelevant_content='0', non_visitor_rant='1', toxicity='0', reasoning="The review is a rant from someone who likely didn't visit the location, repeating a claim from a friend."), MessageInfo(spam='0', advertisement='1', irrelevant_content='0', non_visitor_rant='0', toxicity='0', reasoning='The review promotes Bear Creek Cabins & RV Park, including a discount offer.')])

In [31]:
print(df.head(10).iloc[9]['user_message'])

Business Name: Bear Creek Cabins & RV Park
Category: RV park, Cabin rental agency, Campground
Description: None
Review: Treated terribly by owner absolutely outrageous pricing wouldn't even allow me to shower after paying almost 40.00 for just parking in a lot for 5 hours
Response: None
